In [1]:
import pandas as pd

In [112]:
from geopy.distance import geodesic
import duckdb
from tqdm import tqdm

In [45]:
pd.set_option('display.max_colwidth', None)

In [2]:
rosbank_df=pd.read_csv('Projects/Geo_Atms_Project/rosbank_happy_data_year/data/train.csv')

In [35]:
rosbank_df=rosbank_df.dropna()

In [95]:
def get_region(x: str):
    if x.split(',')[-1].lstrip().isdigit():
        return x.split(',')[-3].lstrip()
    else:
        return x.split(',')[-2].lstrip()

In [96]:
rosbank_df['region']=rosbank_df.address_rus.apply(
    lambda x: get_region(x))

In [97]:
rosbank_df.groupby('region')['id'].count().reset_index(name='cnt_atms').sort_values(by='cnt_atms', ascending=False)

,region,cnt_atms
48,Москва,575
49,Московская область,321
67,Приморский край,206
91,Санкт-Петербург,194
82,Республика Татарстан,179
...,...,...
70,Республика Адыгея,4
69,Псковская область,2
24,Кабардино-Балкарская Республика,1
78,Республика Крым,1


In [119]:
rosbank_df.region.unique()[:5]

array(['Сахалинская область', 'Москва', 'Красноярский край',
       'Приморский край', 'Московская область'], dtype=object)

In [99]:
rosbank_df_mmo=rosbank_df[rosbank_df.region.isin(['Москва', 'Московская область'])]

In [103]:
rosbank_df_mmo.head(2)

,Unnamed: 0,id,atm_group,address,address_rus,lat,long,target,region_dtype,region
3,3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 125315",55.805827,37.515146,-0.094035,1,Москва
6,6,335.0,32.0,"SHUKHOVA, 14 MOSCOW","улица Шухова, 14, Москва, Россия, 115162",55.716859,37.613063,-0.024682,1,Москва


In [88]:
df_mmo_geocoded=pd.read_csv('df_mmo_geocoded.csv')

In [104]:
df_mmo_geocoded.head(2)

,address,address2,geo_lat,geo_lon
0,"г. Сергиев Посад, ул.Кооперативная, д.2","Московская обл, г Сергиев Посад, ул Кооперативная, д 2",56.301963,38.135828
1,"г. Сергиев Посад, ул.Вознесенская, д.32а","Московская обл, г Сергиев Посад, ул Вознесенская, д 32А",56.303856,38.133214


In [107]:
query="""
select
a.*, b.*
from df_mmo_geocoded a 
inner join rosbank_df_mmo b 
    on a.geo_lat between b.lat-0.2 and b.lat+0.2
        and a.geo_lon between b.long-0.1 and b.long+0.1
"""

In [108]:
#Отработали подготовленный выше запрос
df_cross=duckdb.sql(query).df()

In [113]:
#Считаем расстояния
for i, row in tqdm(df_cross.iterrows()):
    df_cross.loc[i, 'dist'] = geodesic((row['geo_lat'], row['geo_lon']), (row['lat'], row['long'])).m

1559906it [02:53, 8997.62it/s]


In [117]:
df_objects=df_cross[df_cross.dist<100][['id','lat','long', 'region']].drop_duplicates()
df_objects.head()

,id,lat,long,region
0,6993.0,56.367123,36.689711,Московская область
15,8382.0,56.339816,38.124635,Московская область
80,6840.0,56.325369,38.132972,Московская область
97,4920.0,56.316129,38.132532,Московская область
120,6026.0,56.316095,38.136844,Московская область


In [118]:
len(df_objects)

610

In [120]:
df_objects.groupby('region')['id'].count().reset_index(name='cnt_atms')

,region,cnt_atms
0,Москва,457
1,Московская область,153
